# Movie Recommendation

## IMDB movie recommendation notebook

#### Author/s: Ruturaj Kiran Vaidya, Brian McClannahan, and Matthew Taylor
(Add you name if you edit/improve)

In [1]:
# Imports

# General

import os
import re
import pandas as pd
import numpy as np
import random
import pickle

# graphs

from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Model
# sklearn
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.feature_extraction import text

In [2]:
from matplotlib import pyplot as plt

In [3]:
train = pd.read_json("MovieDetails.json")
test = pd.read_json("TestMovieDetails.json")

In [4]:
train.keys()

Index(['Actors', 'Awards', 'BoxOffice', 'Country', 'DVD', 'Director',
       'Episode', 'Genre', 'Language', 'Metascore', 'Plot', 'Poster',
       'Production', 'Rated', 'Ratings', 'Released', 'Response', 'Runtime',
       'Season', 'Title', 'Type', 'Website', 'Writer', 'Year', 'imdbID',
       'imdbRating', 'imdbVotes', 'seriesID', 'totalSeasons'],
      dtype='object')

In [5]:
test.keys()

Index(['Actors', 'Awards', 'BoxOffice', 'Country', 'DVD', 'Director',
       'Episode', 'Error', 'Genre', 'Language', 'Metascore', 'Plot', 'Poster',
       'Production', 'Rated', 'Ratings', 'Released', 'Response', 'Runtime',
       'Season', 'Title', 'Type', 'Website', 'Writer', 'Year', 'imdbID',
       'imdbRating', 'imdbVotes', 'seriesID', 'totalSeasons'],
      dtype='object')

In [6]:
del test["Error"]

In [7]:
test.head()

,Actors,Awards,BoxOffice,Country,DVD,Director,Episode,Genre,Language,Metascore,...,Title,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,seriesID,totalSeasons
0,"Kevin Costner, Ashton Kutcher, Sela Ward, Meli...",1 win & 4 nominations.,"$54,983,983",USA,23 Jan 2007,Andrew Davis,NaN,"Action, Adventure, Drama",English,53,...,The Guardian,movie,N/A,Ron L. Brinkerhoff,2006,tt0406816,6.9,"84,017",NaN,NaN
1,"Albert Finney, Tom Courtenay, Edward Fox, Zena...",Nominated for 5 Oscars. Another 5 wins & 12 no...,N/A,UK,06 Apr 2004,Peter Yates,NaN,Drama,English,N/A,...,The Dresser,movie,N/A,"Ronald Harwood (screenplay), Ronald Harwood (b...",1983,tt0085461,7.7,"4,285",NaN,NaN
2,"S. Epatha Merkerson, Marcus Carl Franklin, Yas...",Won 1 Golden Globe. Another 29 wins & 41 nomin...,N/A,USA,23 Aug 2005,George C. Wolfe,NaN,"Drama, Music",English,N/A,...,Lackawanna Blues,movie,N/A,"Ruben Santiago-Hudson (play), Ruben Santiago-H...",2005,tt0407936,7.9,"2,150",NaN,NaN
3,"Julie Andrews, Rock Hudson, Jeremy Kemp, Lance...",Nominated for 3 Oscars. Another 1 win & 4 nomi...,N/A,USA,25 Oct 2005,Blake Edwards,NaN,"Comedy, Drama, Musical, Romance, War","English, German",N/A,...,Darling Lili,movie,N/A,"Blake Edwards, William Peter Blatty",1970,tt0065611,6.1,"1,826",NaN,NaN
4,Mel Blanc,N/A,N/A,USA,N/A,Robert McKimson,NaN,"Family, Animation, Short",English,N/A,...,Acrobatty Bunny,movie,N/A,Warren Foster (story),1946,tt0038286,7.3,663,NaN,NaN


In [8]:
train.head()

,Actors,Awards,BoxOffice,Country,DVD,Director,Episode,Genre,Language,Metascore,...,Title,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,seriesID,totalSeasons
0,"Seth Allen, John Bakos, Mari-Claire Charba, Pe...",N/A,N/A,USA,N/A,Tom O'Horgan,NaN,Comedy,English,N/A,...,Futz,movie,N/A,"Rochelle Owens (play), Joseph Stefano (screenp...",1969,tt0064354,4.8,50,NaN,NaN
1,"Jane Fonda, Robert De Niro, Swoosie Kurtz, Mar...",N/A,N/A,"USA, Canada",13 Jan 2004,Martin Ritt,NaN,"Drama, Romance",English,N/A,...,Stanley & Iris,movie,N/A,"Pat Barker (novel), Harriet Frank Jr. (screenp...",1990,tt0100680,6.3,"6,502",NaN,NaN
2,"Christopher Walken, Jason Connery, Carmela Mar...",N/A,N/A,USA,09 Aug 2005,Eugene Marner,NaN,"Fantasy, Musical, Family",English,N/A,...,Puss in Boots,movie,N/A,"Charles Perrault (fairy tale ""Le chat botté""),...",1988,tt0177606,6.4,537,NaN,NaN
3,N/A,N/A,N/A,N/A,N/A,N/A,NaN,N/A,N/A,N/A,...,#DUPE#,movie,N/A,N/A,1972,tt0068444,N/A,N/A,NaN,NaN
4,"Shemp Howard, Larry Fine, Moe Howard, Christin...",N/A,N/A,USA,N/A,Edward Bernds,NaN,"Comedy, Short",English,N/A,...,Who Done It?,movie,N/A,"Edward Bernds (story), Edward Bernds (screenplay)",1949,tt0042042,8.2,335,NaN,NaN


In [9]:
combined = pd.concat([train, test], ignore_index=True)
movies = combined[["imdbID", "Title", "Ratings", "imdbRating", "imdbVotes", "Language", "Genre"]].dropna()
movies.drop_duplicates(subset=['imdbID'], keep=False, inplace=True)

In [10]:
movies.tail()

,imdbID,Title,Ratings,imdbRating,imdbVotes,Language,Genre
7029,tt0258760,Lammbock,"[{'Source': 'Internet Movie Database', 'Value'...",7.3,"10,180",German,Comedy
7030,tt0146455,Babylon 5: A Call to Arms,"[{'Source': 'Internet Movie Database', 'Value'...",7.2,"4,976",English,"Action, Adventure, Drama, Sci-Fi"
7031,tt0432047,Sarkar,"[{'Source': 'Internet Movie Database', 'Value'...",7.7,"14,199","Hindi, Marathi","Crime, Drama"
7032,tt0109515,Cyber Tracker,"[{'Source': 'Internet Movie Database', 'Value'...",3.5,971,English,"Action, Sci-Fi"
7033,tt0485161,Totally Awesome,"[{'Source': 'Internet Movie Database', 'Value'...",5.2,"1,625",English,Comedy


In [11]:
movies.shape

(6914, 7)

### Encoding genres

In [12]:
# split genres and printing
genres = set([j for i in [i.split(",") for i in movies['Genre']] for j in i])
print(genres)
print(len(genres))

{'Romance', ' Action', 'Talk-Show', ' Sci-Fi', 'Adventure', ' Music', ' Sport', 'Drama', ' News', ' Short', 'News', ' Fantasy', 'War', ' Biography', 'History', ' Romance', ' Adult', 'Game-Show', 'N/A', ' Documentary', ' Comedy', ' Thriller', ' Family', ' War', 'Sport', 'Western', 'Sci-Fi', ' Mystery', ' Crime', 'Film-Noir', ' Game-Show', 'Short', 'Biography', ' Adventure', ' Talk-Show', 'Adult', 'Horror', 'Crime', ' History', ' Drama', ' Musical', 'Thriller', 'Family', ' Western', 'Animation', 'Action', ' Horror', ' Reality-TV', 'Documentary', 'Music', 'Comedy', 'Musical', 'Mystery', 'Reality-TV', ' Animation', 'Fantasy', ' Film-Noir'}
57


In [13]:
# Add columns based on new generes
genres_dic = {j:[1 if j in i else 0 for i in movies["Genre"]] for j in genres}
# Plotting movies per genre
data = [go.Bar(
            x=list(genres_dic.keys()),
            y=[sum(i) for i in genres_dic.values()]
    )]
fig = go.Figure(data=data)
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(
    title="Movies per Genre",
    xaxis_title="Genres",
    yaxis_title="Number of Movies",
)

iplot(fig)
# There is an N/A genre - removing
del genres_dic['N/A']

### Similarly encoding languages

In [14]:
# split genres and printing
languages = set([j for i in [i.split(",") for i in movies['Language']] for j in i])
print(len(languages))

187


In [15]:
# Add columns based on new generes
languages_dic = {j:[1 if j in i else 0 for i in movies["Language"]] for j in languages}
# Plotting movies per genre
data = [go.Bar(
            x=list(languages_dic.keys()),
            y=[sum(i) for i in languages_dic.values()]
    )]
fig = go.Figure(data=data)
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(
    title="Movies per Language",
    xaxis_title="Languages",
    yaxis_title="Number of Movies",
)

iplot(fig)

### Above graph is interactive - and I know there are so many English movies!
### Hence we will not select languages for our analysis

In [16]:
# Now we will add encoded generes and languages
for k,v in genres_dic.items():
    movies[k]=v
#for k,v in languages_dic.items():
#    movies[k]=v
# Dropping rows with no genres listed
movies = movies[movies.Genre != "N/A"]
# We don't need genres column anymore
del movies["Genre"]
del movies["Language"]

# Also converting imdb votes to numeric
movies["imdbVotes"] = movies['imdbVotes'].str.replace(",", "").astype(int)

# Normalize rating
for index, row in movies.iterrows():
    movies.loc[index, 'imdbRating'] = float(row['imdbRating']) / 10

movies = movies.reset_index(drop=True)
    
movies.tail()

,imdbID,Title,Ratings,imdbRating,imdbVotes,Romance,Action,Talk-Show,Sci-Fi,Adventure,...,Reality-TV,Documentary,Music,Comedy,Musical,Mystery,Reality-TV,Animation,Fantasy,Film-Noir
6575,tt0258760,Lammbock,"[{'Source': 'Internet Movie Database', 'Value'...",0.73,10180,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6576,tt0146455,Babylon 5: A Call to Arms,"[{'Source': 'Internet Movie Database', 'Value'...",0.72,4976,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
6577,tt0432047,Sarkar,"[{'Source': 'Internet Movie Database', 'Value'...",0.77,14199,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6578,tt0109515,Cyber Tracker,"[{'Source': 'Internet Movie Database', 'Value'...",0.35,971,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6579,tt0485161,Totally Awesome,"[{'Source': 'Internet Movie Database', 'Value'...",0.52,1625,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Text Vectorization

In [17]:
# Instantiate vectorizers
stop_words = text.ENGLISH_STOP_WORDS.union(['movie', 'film', 'like', 'love', 'just', 'good', 'time', 'really',
                                            'story', 'bad', 'movies', 'don', 'people', 'make', 'great',
                                            'way', 'characters', 'character', 'watch', 'films', 'acting',
                                            'seen', 'think', 'does', 'little', 'plot', 'does', 'did',
                                            've' 'doesn', 'know', 'life', 'end', 'scene', 'man', 've',
                                            'say', 'thing', 'didn', 'things', 'better', 'best', 'scenes'])
count_vectorizer = text.CountVectorizer(stop_words=stop_words)
tfidf_vectorizer = text.TfidfVectorizer(stop_words=stop_words)

In [18]:
# Get the movie ids of each review
# Note: You must place the "test" and "train" directories from the dataset
#       available here: http://ai.stanford.edu/~amaas/data/sentiment/ in 
#       a directory named data/raw for these cells to work
test_neg_movie_ids = [x.split('/')[4] for x in open('../../data/raw/test/urls_neg.txt').readlines()]
test_pos_movie_ids = [x.split('/')[4] for x in open('../../data/raw/test/urls_pos.txt').readlines()]
train_neg_movie_ids = [x.split('/')[4] for x in open('../../data/raw/train/urls_neg.txt').readlines()]
train_pos_movie_ids = [x.split('/')[4] for x in open('../../data/raw/train/urls_pos.txt').readlines()]

In [19]:
# master list of all reviews, to be appended to movies dataframe
reviews = [[] for _ in range(len(movies))]

# regex to remove html from reviews
html_regex = re.compile(r'<[^>]+>')

print('Loading reviews...', flush=True)

# get the names of files containing reviews
test_neg_review_files = os.listdir('../../data/raw/test/neg')
for i in range(len(test_neg_movie_ids)):
    
    movie_id = test_neg_movie_ids[i]
    
    # ignore reviews for movies that have removed from the dataframe
    if movie_id not in movies.imdbID.values:
        continue
        
    # get the movie's dataframe index
    index = movies.loc[movies.imdbID == movie_id].index[0]
    
    # load review and remove html
    review_text = open('../../data/raw/test/neg/{}'.format(test_neg_review_files[i]), encoding='utf8').read()
    review_text = html_regex.sub(' ', review_text)
    
    # assign review to movie
    reviews[index].append(review_text)
    
test_pos_review_files = os.listdir('../../data/raw/test/pos')
for i in range(len(test_pos_movie_ids)):
    
    movie_id = test_pos_movie_ids[i]
    
    # ignore reviews for movies that have removed from the dataframe
    if movie_id not in movies.imdbID.values:
        continue
        
    # get the movie's dataframe index
    index = movies.loc[movies.imdbID == movie_id].index[0]
    
    # load review and remove html
    review_text = open('../../data/raw/test/pos/{}'.format(test_pos_review_files[i]), encoding='utf8').read()
    review_text = html_regex.sub(' ', review_text)
    
    # assign review to movie
    reviews[index].append(review_text)
    
train_neg_review_files = os.listdir('../../data/raw/train/neg')
for i in range(len(train_neg_movie_ids)):
    
    movie_id = train_neg_movie_ids[i]
    
    # ignore reviews for movies that have removed from the dataframe
    if movie_id not in movies.imdbID.values:
        continue
        
    # get the movie's dataframe index
    index = movies.loc[movies.imdbID == movie_id].index[0]
    
    # load review and remove html
    review_text = open('../../data/raw/train/neg/{}'.format(train_neg_review_files[i]), encoding='utf8').read()
    review_text = html_regex.sub(' ', review_text)
    
    # assign review to movie
    reviews[index].append(review_text)
    
train_pos_review_files = os.listdir('../../data/raw/train/pos')
for i in range(len(train_pos_movie_ids)):
    
    movie_id = train_pos_movie_ids[i]
    
    # ignore reviews for movies that have removed from the dataframe
    if movie_id not in movies.imdbID.values:
        continue
        
    # get the movie's dataframe index
    index = movies.loc[movies.imdbID == movie_id].index[0]
    
    # load review and remove html
    review_text = open('../../data/raw/train/pos/{}'.format(train_pos_review_files[i]), encoding='utf8').read()
    review_text = html_regex.sub(' ', review_text)
    
    # assign review to movie
    reviews[index].append(review_text)
    
print('Done', flush=True)

Loading reviews...
Done


In [20]:
# concatenate results, append to dataframe
reviews = [' '.join(i) for i in reviews]
movies['reviews'] = reviews

In [21]:
# vectorize reviews
count_vectors = count_vectorizer.fit_transform(reviews)
tfidf_vectors = tfidf_vectorizer.fit_transform(reviews)

### Using K-means clustering

In [22]:
# Here I am selecting ratings, votes, genres and languages to train
X = movies.drop(columns=['imdbVotes', 'reviews']).replace([np.inf, -np.inf], np.nan).dropna().iloc[:,3:].to_numpy()
# I am selecting 30 clusters based on number of languages, but this can be changed later
# I roughly selected the number of clusters, but this can be changed for e.g. based on genres
kmeans = KMeans(n_clusters=30, random_state=4).fit(X)

# create kmeans models based on reviews
if os.path.exists('pickle/kmeans_count'):
    kmeans_count = pickle.load(open('pickle/kmeans_count', 'rb'))
else:
    
    if not os.path.exists('pickle'):
        os.mkdir('pickle')
        
    kmeans_count = KMeans(n_clusters=8, random_state=4).fit(count_vectors)
    pickle.dump(kmeans_count, open('pickle/kmeans_count', 'wb'))
    
if os.path.exists('pickle/kmeans_tfidf'):
    kmeans_tfidf = pickle.load(open('pickle/kmeans_tfidf', 'rb'))
else:
    kmeans_tfidf = KMeans(n_clusters=8, random_state=4).fit(tfidf_vectors)
    pickle.dump(kmeans_tfidf, open('pickle/kmeans_tfidf', 'wb'))

### Using DBSCAN

In [23]:
dbscan = DBSCAN(min_samples=10).fit(X)

In [24]:
print(np.unique(dbscan.labels_))
dbscan_dic = {i:[] for i in dbscan.labels_}
[dbscan_dic[i].append(list(movies["Title"])[c]) for c, i in enumerate(dbscan.labels_)]
print([len(i) for i in dbscan_dic.values()])

[ -1   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114]
[363, 245, 2127, 32, 96, 51, 21, 42, 36, 81, 22, 89, 36, 573, 16, 156, 23, 48, 30, 71, 10, 216, 16, 14, 23, 16, 34, 16, 20, 137, 33, 13, 15, 18, 64, 43, 10, 94, 12, 17, 14, 14, 15, 175, 22, 15, 19, 10, 33, 16, 14, 37, 20, 18, 19, 15, 12, 33, 11, 20, 22, 23, 22, 58, 10, 12, 38, 15, 19, 27, 19, 23, 17, 35, 10, 13, 40, 10, 16, 17, 10, 16, 10, 16, 16, 28, 20, 10, 33, 16, 47, 15, 10, 16, 12, 32, 12, 49, 12, 26, 18, 14, 15, 13, 12, 16, 10, 15, 16, 14, 12, 18, 11, 13, 10, 10]


Looks like there are too many outlier movies for dbscan to work.

### Agglomerative Clustering

In [25]:
ac = AgglomerativeClustering(n_clusters = 30).fit(X)

In [26]:
print(np.unique(ac.labels_))
ac_dic = {i:[] for i in ac.labels_}
[ac_dic[i].append(list(movies["Title"])[c]) for c, i in enumerate(ac.labels_)]
print([len(i) for i in ac_dic.values()])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[374, 323, 226, 180, 217, 125, 304, 186, 253, 274, 341, 155, 128, 573, 194, 182, 110, 71, 209, 408, 356, 224, 128, 192, 166, 103, 222, 125, 180, 51]


### Recommendation Function

In [27]:
comb_dic = {i:[] for i in kmeans.labels_}
[comb_dic[i].append(list(movies["Title"])[c]) for c, i in enumerate(kmeans.labels_)]
print([len(i) for i in comb_dic.values()])

[633, 363, 182, 141, 283, 213, 340, 270, 286, 215, 55, 172, 188, 625, 264, 241, 271, 174, 232, 116, 233, 170, 109, 129, 159, 114, 109, 86, 113, 94]


In [28]:
# Simplest function - note that it may give same value as recommendation as it randomly selects the movie
# This can be changed later
def movie_recommendation(x,model,dic,df):
    return random.choice(dic[model.labels_[list(df["Title"]).index(x)]])

In [29]:
def all_movie_recommendations(x,model,dic,df):
    return dic[model.labels_[list(df["Title"]).index(x)]]

In [30]:
#Recommends movies based on the intersection of all three clustering methods
def combined_approach(k_rec,d_rec,a_rec):
    combo_list = [value for value in k_rec if value in d_rec]
    combo_list = [value for value in combo_list if value in a_rec]
    return combo_list

In [31]:
def compare_techniques(movie):
    print("Comparing techniques on movie", movie, "with genres", combined[combined.Title == movie].Genre.values)
    k_list = all_movie_recommendations(movie,kmeans,comb_dic,movies)
    d_list = all_movie_recommendations(movie,dbscan,dbscan_dic,movies)
    a_list = all_movie_recommendations(movie,ac,ac_dic,movies)
    kmeans_rec = random.choice(k_list)
    dbscan_rec = random.choice(d_list)
    ac_rec = random.choice(a_list)
    combo_rec = random.choice(combined_approach(k_list,d_list,a_list))
    print("Kmeans recommends", kmeans_rec, "with genres", combined[combined.Title == kmeans_rec].Genre.values)
    print("DBSCAN recommends", dbscan_rec, "with genres", combined[combined.Title == dbscan_rec].Genre.values)
    print("AC recommends", ac_rec, "with genres", combined[combined.Title == ac_rec].Genre.values)
    print("Combo recommends", combo_rec, "with genres", combined[combined.Title == combo_rec].Genre.values)

In [32]:
movie = "Totally Awesome"
kmeans_rec = all_movie_recommendations(movie,kmeans,comb_dic,movies)
dbscan_rec = all_movie_recommendations(movie,dbscan,dbscan_dic,movies)
ac_rec = all_movie_recommendations(movie,ac,ac_dic,movies)
print(len(kmeans_rec), len(dbscan_rec), len(ac_rec))
c = combined_approach(kmeans_rec,dbscan_rec,ac_rec)
print(len(c))

633 363 374
363


In [33]:
compare_techniques("Totally Awesome")

Comparing techniques on movie Totally Awesome with genres ['Comedy']
Kmeans recommends The Groove Tube with genres ['Comedy']
DBSCAN recommends Samantha with genres ['Comedy']
AC recommends Last Call with Carson Daly with genres ['Comedy, Music, Talk-Show']
Combo recommends Can I Do It 'Till I Need Glasses? with genres ['Comedy']


In [34]:
compare_techniques("Star Wars: Episode VI - Return of the Jedi")

Comparing techniques on movie Star Wars: Episode VI - Return of the Jedi with genres ['Action, Adventure, Fantasy, Sci-Fi']
Kmeans recommends Kirby: Right Back at Ya! with genres ['Animation, Action, Adventure, Comedy, Family, Fantasy, Sci-Fi']
DBSCAN recommends The Great Rock 'n' Roll Swindle with genres ['Music']
AC recommends The Savage Curtain with genres ['Action, Adventure, Mystery']
Combo recommends The Omega Code with genres ['Action, Adventure, Fantasy, Sci-Fi, Thriller']


In [35]:
compare_techniques("Good Will Hunting")

Comparing techniques on movie Good Will Hunting with genres ['Drama, Romance']
Kmeans recommends Fraulein Doktor with genres ['Drama, History, Romance']
DBSCAN recommends Of Human Bondage with genres ['Drama, Romance']
AC recommends Angels Wash Their Faces with genres ['Drama, Romance']
Combo recommends The Cooler with genres ['Drama, Romance']


In [36]:
# Record clusters for each movie, to aid recommendation function

kmeans_count_clusters = []
kmeans_tfidf_clusters = []

kmeans_count.predict(count_vectorizer.transform([movies.reviews.values[0]]))

for r in movies.reviews.values:
    c = count_vectorizer.transform([r])
    t = tfidf_vectorizer.transform([r])
    
    kmeans_count_clusters.append(kmeans_count.predict(c)[0])
    kmeans_tfidf_clusters.append(kmeans_tfidf.predict(t)[0])
    
movies['kmeans_count_cluster'] = kmeans_count_clusters
movies['kmeans_tfidf_cluster'] = kmeans_tfidf_clusters

Here, we define a function that recommends movies based on reviews on those movies. It works by randomly selecting three other movies from the cluster that the given movie belongs to.

In [37]:
# Review recommendation function declaration
def recommend_by_reviews(movie_title):
    
    c_cluster = movies.loc[movies.Title == movie_title].kmeans_count_cluster.values[0]
    t_cluster = movies.loc[movies.Title == movie_title].kmeans_tfidf_cluster.values[0]
    
    possible_count_recommendations = list(movies.loc[movies.kmeans_count_cluster == c_cluster].Title.values)
    possible_tfidf_recommendations = list(movies.loc[movies.kmeans_tfidf_cluster == t_cluster].Title.values)
    
    possible_count_recommendations.remove(movie_title)
    possible_tfidf_recommendations.remove(movie_title)
    
    c = random.sample(possible_count_recommendations, 3)
    t = random.sample(possible_tfidf_recommendations, 3)
    
    print('Finding recommendations for "{}", (count cluster: {}, tfidf cluster: {})'.format(movie_title, c_cluster, t_cluster))
    print('Count vectorizer recommends "{}", "{}", and "{}"'.format(c[0], c[1], c[2]))
    print('Tfidf vectorizer recommends "{}", "{}", and "{}"'.format(t[0], t[1], t[2]))

Before we check the recommendations, it's worth examining the size/distribution of each of the clusters. Doing this shows that the count vectorizer is highly non-uniform. Over half of the movies are placed in a single cluster. The TFIDF vectorizer, on the other hand, is much more uniform.

In [38]:
c = [0] * 8
t = [0] * 8

for i in list(movies.kmeans_count_cluster.values):
    c[i] += 1
    
for i in list(movies.kmeans_tfidf_cluster.values):
    t[i] += 1

print('Movies per cluster for count vectorizer:')
print(c)
print('\nMovies per cluster for tfidf vectorizer:')
print(t)

Movies per cluster for count vectorizer:
[21, 194, 1, 5, 237, 1492, 569, 4061]

Movies per cluster for tfidf vectorizer:
[89, 1712, 275, 94, 142, 26, 1203, 3039]


To further examine the contents of the clusters, we can print the words whose encodings are the closest to each of the cluster centers. This shows that many clusters have their own distinct characteristics, while other cluster centers are rather close to each other.

In [39]:
print('Top 10 words in each count vectorizer cluster:')
count_centers = kmeans_count.cluster_centers_.argsort()[:, ::-1]
word_encodings = count_vectorizer.get_feature_names()

for i in range(8):
    print('Cluster {}: '.format(i), end='')
    for j in count_centers[i, :9]:
        print('{}, '.format(word_encodings[j]), end='')
    print(word_encodings[count_centers[i, 9]])

print('\nTop 10 words in each tfidf vectorizer cluster:')
tfidf_centers = kmeans_tfidf.cluster_centers_.argsort()[:, ::-1]
word_encodings = tfidf_vectorizer.get_feature_names()

for i in range(8):
    print('Cluster {}: '.format(i), end='')
    for j in tfidf_centers[i, :9]:
        print('{}, '.format(word_encodings[j]), end='')
    print(word_encodings[tfidf_centers[i, 9]])
    
# make room for visualizations
del test_neg_movie_ids
del test_pos_movie_ids
del train_neg_movie_ids
del train_pos_movie_ids
del kmeans_count
del kmeans_tfidf
del word_encodings

Top 10 words in each count vectorizer cluster:
Cluster 0: zombie, zombies, dead, horror, video, music, actually, action, watching, doesn
Cluster 1: horror, 10, doesn, actually, director, watching, real, old, new, lot
Cluster 2: cinderella, disney, fairy, mice, prince, ball, step, classic, stepmother, mother
Cluster 3: davis, mildred, jane, hospital, bette, years, howard, version, philip, performance
Cluster 4: years, real, world, young, work, new, makes, old, war, series
Cluster 5: real, watching, doesn, actors, old, director, going, funny, actually, work
Cluster 6: funny, real, actors, doesn, watching, years, work, actually, director, makes
Cluster 7: watching, funny, actors, actually, 10, director, doesn, real, old, look

Top 10 words in each tfidf vectorizer cluster:
Cluster 0: zombie, seagal, zombies, steven, dead, action, horror, gore, worst, budget
Cluster 1: real, actors, years, series, old, director, funny, watching, work, 10
Cluster 2: funny, comedy, jokes, humor, laugh, sandl

Some of the most distinct clusters are those that describe horror movies. Recommending those may yield the best results. As shown below, recommendations for a movie called 'The Zombie Chronicles' include 'A Nightmare on Elm Street 5' and 'The Manson Family', which seem appropriate.

In [56]:
recommend_by_reviews('The Zombie Chronicles')

Finding recommendations for "The Zombie Chronicles", (count cluster: 0, tfidf cluster: 0)
Count vectorizer recommends "A Nightmare on Elm Street 5: The Dream Child", "A Chinese Ghost Story", and "They All Laughed"
Tfidf vectorizer recommends "The Captives", "Nothing", and "The Manson Family"


Other kinds of movies yield less impressive results. The count vectorizer results for a movie called 'A Little Princess' seem like they have nothing to do with that movie. Inspection reveals that it belongs to the mega-cluster present in the count vectorizer clustering algorithm. The TFIDF vectorizer mentions a movie named 'Penny Princess', which is on the right track, though the other two movies are nothing like the given movie.

In [99]:
recommend_by_reviews('A Little Princess')

Finding recommendations for "A Little Princess", (count cluster: 7, tfidf cluster: 3)
Count vectorizer recommends "Moby Dick", "The Sissy Duckling", and "Brief Encounter"
Tfidf vectorizer recommends "The Act of Seeing with One's Own Eyes", "Penny Princess", and "Algie, the Miner"


Going back to the contents of the reviews for these movies reveals something worth noting. Many reviews contain extremely similar wording. Most reviews are long the lines of 'This movie was great, I loved it! The actors were so believable'. While this is obviously a glowing review and can help a sentiment analysis tool learn the meaning of the words 'great' and 'loved', it does little to make movie recommendations, considering this review could potentially be given to a movie of any kind.

## Visualizations

In [43]:
df_pie_visualization = movies.drop(columns=["Title","imdbID","Ratings","imdbRating","imdbVotes", "reviews", "kmeans_count_cluster", "kmeans_tfidf_cluster"])
df_pie_visualization

,Romance,Action,Talk-Show,Sci-Fi,Adventure,Music,Sport,Drama,News,Short,...,Reality-TV,Documentary,Music,Comedy,Musical,Mystery,Reality-TV,Animation,Fantasy,Film-Noir
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
dict_genres = dict()
total = 0
for column in df_pie_visualization.columns:
    count = df_pie_visualization[column].sum()
    total += count
    dict_genres[column] = count

In [45]:
data = []
labels = []
for key in dict_genres.keys():
    labels.append(key)
    data.append(dict_genres[key]/total)
label_count = len(labels)
other = 0
i = 0
while i < len(data):
    if data[i] < .02:
        other += data[i]
        del data[i]
        del labels[i]
    else:
        i += 1
print(sum(data))
print(data)
for i in range(len(data)):
    data[i] = data[i]/sum(data)
print(sum(data))
print(data)

0.7225562750539621
[0.046137835337650326, 0.022664199814986123, 0.0274822695035461, 0.12068300955905026, 0.04459605303731113, 0.02135368485969781, 0.04555966697502312, 0.024707061362935552, 0.020004625346901016, 0.03746530989824237, 0.034728646315140305, 0.06355997533148319, 0.04925994449583719, 0.041666666666666664, 0.08001850138760407, 0.022355843354918286, 0.020312981806968857]
0.8899541528977284
[0.06385362210604932, 0.030616041025771847, 0.03673000741106852, 0.15932346907295378, 0.056017303356960896, 0.026443099643160207, 0.056064965081007216, 0.0300160600198305, 0.02414742924569956, 0.04499905825055258, 0.041338035371378326, 0.07506584214572062, 0.05739721208686339, 0.04809360019686178, 0.09168095393364298, 0.02527639066693514, 0.022891063283271673]


In [46]:
fig_pie,ax_pie = plt.subplots(figsize=(11,8))
plt.pie(data,radius=4.0,labels=labels)
plt.axis('equal')
plt.tight_layout()
plt.show()